In [1]:
# from fraction_general import TSSs_l
# from matplotlib.lines import _LineStyle
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats

dic = {
    'TSSs_l':'./data/gene/low_expressed.bed',
    'location':'./data/gene/GRCh37.gene.bed',
    'bam':'./data/051.bam',
    'test_bam':'./data/051_2.bam',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed'
    }

chr_to_id = {
    'chr1':'1','chr2':'2','chr3':'3','chr4':'4','chr5':'5','chr6':'6','chr7':'7','chr8':'8','chr9':'9','chr10':'10','chr11':'11','chr12':'12','chr13':'13','chr14':'14','chr15':'15','chr16':'16','chr17':'17','chr18':'18','chr19':'19','chr20':'20','chr21':'21','chr22':'22','chrX':'X','chrY':'Y'
}

up = 1000
down = 1000

bamfile = ps.AlignmentFile(dic['bam'],'rb')

# TSSs_l = []
# with open(dic['TSSs_l'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         if ll[0] == 'chr1':
#             TSSs_l.append(TSS(chr_to_id[ll[0]],int(ll[1])))
TSS_silent = []
with open(dic['TSS_silent'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '1':
            TSS_silent.append(TSS(ll[0], int(ll[1])+int(1000)))

TSS_HK = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '1':
            TSS_HK.append(TSS(ll[0], int(ll[1])+int(1000)))

test_HK = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '2':
            test_HK.append(TSS(ll[0], int(ll[1])+int(1000)))

# p = 25
# TSS_silent = TSS_silent[0:p]
# TSS_HK = TSS_HK[0:p]

win = 200
TSSs_l = TSS_HK
# coverage_array = np.zeros((len(TSSs_l), int(up+down)), dtype=int)
long_array = np.zeros((len(TSSs_l), int(up+down)), dtype=int)
short_array = np.zeros((len(TSSs_l), int(up+down)), dtype=int)
for j, tss in enumerate(TSSs_l):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    
    plist = []
    for r in bamfile.fetch(chrom, start, end):
         if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            ss = max(0, r.reference_start - start)
            ee = min(r.reference_start - start + abs(r.isize), end - start)
            if r.isize <= 150:
                for i in range(ss, ee):
                    short_array[j][i] += 1
            if r.isize > 150:
                for i in range(ss, ee):
                    long_array[j][i] += 1
ref_x_minus_y = [np.mean(long_array[:,i])-np.mean(short_array[:,i]) for i in range(up+down)]



In [ ]:
# for j, tss in enumerate(TSS_HK):
#     chrom = tss.chrom
#     start = tss.pos - up
#     end = tss.pos + down
#     plist = []
#     for r in bamfile.fetch(chrom, start, end):
#          if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
#             ss = max(0, r.reference_start - start)
#             ee = min(r.reference_start - start + abs(r.isize), end - start)
#             if r.isize <= 150:
#                 for i in range(ss, ee):
#                     short_array[j][i] += 1
#             if r.isize > 150:
#                 for i in range(ss, ee):
#                     long_array[j][i] += 1

In [ ]:
import numpy as np

a = np.array([1,2,3,4])
b = np.array([1,2,4,8])
# print(a-b)
import pandas as pd
dataframe = pd.DataFrame({'long_array':a,'short_array':b})
print(dataframe)

In [18]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import sys
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
df.head()
# print(df.head())
 
train, test = df[df['is_train']==True], df[df['is_train']==False]
 
features = df.columns[:4]
clf = RandomForestClassifier(n_jobs=2)
y, _ = pd.factorize(train['species'])
print(train[features])
# print(y)
sys.exit()
clf.fit(train[features], y)
 
preds = iris.target_names[clf.predict(test[features])]
pd.crosstab(test['species'], preds, rownames=['actual'], colnames=['preds'])

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
6                  4.6               3.4                1.4               0.3
..                 ...               ...                ...               ...
142                5.8               2.7                5.1               1.9
143                6.8               3.2                5.9               2.3
144                6.7               3.3                5.7               2.5
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[101 rows x 4 columns]


SystemExit: 

/home/jiay/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
